In [2]:
"""
Train a custom CNN network using CIFAR100 dataset.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os.path
from torchvision import models

#from prep import get_data,beavernet_transform,alexnet_transform

class BeaverNet(nn.Module):
    """
    Since training AlexNet is time consuming,
    we will use a much simpler CNN architecture.
    """
    def __init__(self, num_classes=100):
        super(BeaverNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train_nn(net, epochs, trainloader, loss_function, optimizer):
    """
    Train net epochs number of times using data from trainloader
    and use loss_function and optimizer to get better.
    """
    for epoch in range(epochs):
        print('Epoch:', epoch+1)
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # Get one batch of both images and labels.
            images, classes = data

            # Forward pass: predict classes for a given image.
            outputs = net(images)

            # Calculate the difference between
            # what we've predicted and what we should
            # predict.
            loss = loss_function(outputs, classes)

            # Because changes('gradients') are accumulated
            # from one iteration to another we need to
            # clean up the last ones, so we can propagate
            # the ones from this iteration.
            # Note: always call it before
            # loss.backward() and optimizer.step()
            optimizer.zero_grad()

            # Backward pass: accumulate changes('gradients')
            # that we've learned about in this iteration.
            loss.backward()

            # Backward pass: propagate changes trough the network.
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:
                print('[%d, %d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Trained on %d images' % i)

def test_nn(net, testloader, classes, batch_size=4):
    """
    Quickly test net on a small amount of data.
    """
    # Get the first image from a test data set
    dataiter = iter(testloader)
    images, labels = dataiter.next()
    print('Trying to predict ')
    print(' '.join(['%s' % classes[labels[j]] for j in range(batch_size)]))
    # Feed the image to the network and
    # get the classified classes.
    outputs = net(images)
    # Get the most probable classes first.
    _, predicted = torch.max(outputs, 1)
    print('Predicted: ')
    print(' '.join(['%s' % classes[predicted[j]] for j in range(batch_size)]))

def test_nn_all(net, testloader):
    """
    Test data on all test dataset, calculate how
    much images have been classified correctly.
    """
    correct = 0
    total = 0
    # When testing we don't need to adjust
    # the network parameters, so we can
    # turn off accumulating changes('gradients').
    # (this will save us memory)
    with torch.no_grad():
        for i, data in enumerate(testloader):
            # Get a single batch of images
            # and associated classes.
            images, classes = data
            # Feed the network with those images
            # to check how they will be classified.
            outputs = net(images)
            # Get the most probable classes first.
            _, predicted = torch.max(outputs.data, 1)
            # Add current number images we process to the total.
            total += len(images)
            # How much images were classified correctly?
            correct += (predicted == classes).sum().item()

    print('Test accuracy on %d test images: %d %%' % (i, 100 * correct / total))


In [3]:
"""
Get and prepare the CIFAR100 dataset and its classes
to train a custom CNN Beaver detector.
"""
import torch
torch.manual_seed(1)

from torchvision import transforms, utils
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pickle

def get_data(batch_size=4, transform=None):
    """
    Get training and test sets ready to use
    with our network.
    Return also the class names/labels that are
    available in our dataset.
    batch_size - a number of samples to split our dataset into
    transform - transform.Compose, list of transforms to do on each image
    """
    trainset = CIFAR100(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=2)

    testset = CIFAR100(root='./data', train=False, download=True, transform=transform)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    # After data has been downloaded we can access the mapping
    # between class indexes and the name of the classes.
    classes=pickle.load(open('./data/cifar-100-python/meta', 'rb'))
    classes=classes['fine_label_names']

    return trainloader, testloader, classes

def imshow(img):
    """
    Show images.
    """
    # Since we've already transformed
    # our images we need to "undo" it
    # simple to make them more visible.
    img = img / 2 + 0.5
    npimg = img.numpy()
    npimg=np.transpose(npimg, (1, 2, 0))
    plt.imshow(npimg)
    plt.show()

def show(trainloader,classes,batch_size=4):
    """
    Show some images from training set.
    """
    # Turn data loader to interator,so
    # we can get some images.
    dataiter = iter(trainloader)
    # Get a single batch (4 images).
    images, labels = dataiter.next()
    # Show the name of the classes for those images.
    print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))
    # Show images.
    imshow(utils.make_grid(images))

# The output of torchvision datasets are PILImage images of range [0, 1].
# We transform them to Tensors of normalized range [-1, 1].
beavernet_transform = transforms.Compose([
 transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

alexnet_transform = transforms.Compose([
 transforms.Resize(226),
 transforms.CenterCrop(224),
 transforms.ToTensor(),
 transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
if __name__ == '__main__':
    from sys import argv
    epochs=int(argv[1])
    # To train models defined in pytorch
    # use get_data(transform=alexnet_transform)
    # and get_nn(models.alexnet(num_classes=100))
    train, test, classes=get_data(transform=beavernet_transform)
    net=BeaverNet(num_classes=100)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters())
    train_nn(net, epochs, train, loss_function, optimizer)
    test_nn(net, test, classes)
    test_nn_all(net, test)
    torch.save(net.state_dict(), 'model.ckpt')
    print('Model saved in model.ckpt')

In [7]:
epochs=5

In [8]:
train, test, classes=get_data(transform=beavernet_transform)

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
net=BeaverNet(num_classes=100)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [10]:
train_nn(net, epochs, train, loss_function, optimizer)

Epoch: 1
[1, 2000] loss: 4.391
[1, 4000] loss: 4.058
[1, 6000] loss: 3.936
[1, 8000] loss: 3.814
[1, 10000] loss: 3.758
[1, 12000] loss: 3.725
Epoch: 2
[2, 2000] loss: 3.686
[2, 4000] loss: 3.616
[2, 6000] loss: 3.600
[2, 8000] loss: 3.532
[2, 10000] loss: 3.506
[2, 12000] loss: 3.500
Epoch: 3
[3, 2000] loss: 3.480
[3, 4000] loss: 3.430
[3, 6000] loss: 3.428
[3, 8000] loss: 3.368
[3, 10000] loss: 3.357
[3, 12000] loss: 3.356
Epoch: 4
[4, 2000] loss: 3.346
[4, 4000] loss: 3.323
[4, 6000] loss: 3.325
[4, 8000] loss: 3.260
[4, 10000] loss: 3.256
[4, 12000] loss: 3.269
Epoch: 5
[5, 2000] loss: 3.252
[5, 4000] loss: 3.243
[5, 6000] loss: 3.250
[5, 8000] loss: 3.199
[5, 10000] loss: 3.198
[5, 12000] loss: 3.199
Trained on 12499 images


In [11]:
test_nn(net, test, classes)
test_nn_all(net, test)

Trying to predict 
mountain forest seal mushroom
Predicted: 
skyscraper shrew motorcycle shark
Test accuracy on 2499 test images: 20 %


In [ ]:
torch.save(net.state_dict(), 'model.ckpt')
print('Model saved in model.ckpt')

In [12]:
classes

['apple',
 'aquarium_fish',
 'baby',
 'bear',
 'beaver',
 'bed',
 'bee',
 'beetle',
 'bicycle',
 'bottle',
 'bowl',
 'boy',
 'bridge',
 'bus',
 'butterfly',
 'camel',
 'can',
 'castle',
 'caterpillar',
 'cattle',
 'chair',
 'chimpanzee',
 'clock',
 'cloud',
 'cockroach',
 'couch',
 'crab',
 'crocodile',
 'cup',
 'dinosaur',
 'dolphin',
 'elephant',
 'flatfish',
 'forest',
 'fox',
 'girl',
 'hamster',
 'house',
 'kangaroo',
 'keyboard',
 'lamp',
 'lawn_mower',
 'leopard',
 'lion',
 'lizard',
 'lobster',
 'man',
 'maple_tree',
 'motorcycle',
 'mountain',
 'mouse',
 'mushroom',
 'oak_tree',
 'orange',
 'orchid',
 'otter',
 'palm_tree',
 'pear',
 'pickup_truck',
 'pine_tree',
 'plain',
 'plate',
 'poppy',
 'porcupine',
 'possum',
 'rabbit',
 'raccoon',
 'ray',
 'road',
 'rocket',
 'rose',
 'sea',
 'seal',
 'shark',
 'shrew',
 'skunk',
 'skyscraper',
 'snail',
 'snake',
 'spider',
 'squirrel',
 'streetcar',
 'sunflower',
 'sweet_pepper',
 'table',
 'tank',
 'telephone',
 'television',
 'tig